# **Progetto di Social Computing**


## Librerie

Importiamo le seguenti librerie per realizzare gli obiettivi del progetto.


In [ ]:
!pip install pyvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.7 MB/s eta 0:00:00


In [ ]:
import json
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import random
import requests
import scipy

from copy import deepcopy
from pyvis.network import Network

## Caricare in memoria i dati serializzati relativi alle dipendenze del pacchetto scelto

Prendiamo il pacchetto [xo](https://www.npmjs.com/package/xo?activeTab=readme) e, sfruttando l'API ufficiale di NPM, scarichiamo le sue dipendenze dirette e indirette.

In [ ]:
def _get_dependencies(package: str) -> dict:
    """
    Scarica le dipendenze dirette di un pacchetto usando l'API di NPM,
      documentata al seguente link:
    https://github.com/npm/registry/blob/main/docs/REGISTRY-API.md.

    Args:
        package (str): Nome del pacchetto.

    Returns:
        dict: Dizionario contenente le dipendenze dirette del package.
    """
    url = f"https://registry.npmjs.org/{package}/latest"
    my_request = requests.get(url)
    data = my_request.json()
    try:
        dependencies = data["dependencies"]
        return dependencies
    except KeyError:
        return {}

def find_dependencies(package: str, dependencies_map: dict = None) -> dict:
    """
    Ricerca le sotto-dipendenze di un pacchetto.

    Args:
        package (str): Il nome del pacchetto.
        dependencies_map (dict, optional): Dizionario contenente le dipendenze dirette
            del pacchetto. Se non fornito, verrà utilizzato un dizionario vuoto.

    Returns:
        dict: Dizionario contenente tutte le dipendenze del pacchetto,
          incluse le sotto-dipendenze.
    """
    if dependencies_map is None:
        dependencies_map = {}

    to_process = [package]    # lista per gestire i pacchetti da elaborare

    while to_process:
        current_package = to_process.pop(0)

        if current_package in dependencies_map:
            continue

        try:
            dependencies = _get_dependencies(current_package)
            dependencies_map[current_package] = list(dependencies.keys())
            for dep in dependencies:
                if dep not in dependencies_map:
                    to_process.append(dep)
        except KeyError:
            dependencies_map[package] = []

    return dependencies_map

In [ ]:
package = "xo"

# Dizionario che rappresenta il grafo orientato delle dipendenze
dependencies_map = find_dependencies(package)

A questo punto, salviamo in un file JSON le dipendenze del pacchetto.

In [ ]:
def save_in_file(file_name: str, data_to_save: dict) -> None:
    """
    Salva le dipendenze in un file JSON.

    Args:
        file_name (str): Percorso al file JSON in cui salvare le dipendenze.
        data_to_save (dict): Dizionario contenente le dipendenze.

    Returns:
        None: La funzione scrive il file JSON.
    """
    try:
        with open(file_name, "w") as f:
            json.dump(data_to_save, f, indent = 4)
        print(f"{file_name} salvato!")
    except Exception:
        print("File non salvato")

save_in_file("data/data.json", dependencies_map)

File non salvato


## Utilizzando NetworkX, costruire il grafo diretto

* Ciascun nodo è un pacchetto da cui il pacchetto seed dipende.
* Ogni arco indica una relazione di dipendenza tra due pacchetti.
* Ogni sotto dipendenza è inclusa nel grafo, fino al livello più profondo.

In [ ]:
def build_graph(dependencies_map: dict) -> nx.DiGraph:
    """
    Costruisce il grafo diretto delle dipendenze,
      a partire da un dizionario delle dipendenze.

    Args:
        dependencies_map (dict): dizionario delle dipendenze.

    Returns:
        nx.DiGraph: grafo orientato rappresentante le dipendenze.
    """
    G = nx.DiGraph()
    for package, dependencies in dependencies_map.items():
        for dep in dependencies:
            G.add_edge(package, dep)
    return G

directed_graph = build_graph(dependencies_map)

## Successivamente, costruire un secondo grafo

* Il grafo prodotto al punto precedente viene trasformato in indiretto.
* Il grafo indiretto viene aumentato utilizzando la tecnica del preferential attachment:
  * Scegliendo m, aggiungere almeno altri 350 nodi e 350*m archi.

In [ ]:
def graph_expand_preferential_attachment(undirected_graph: nx.Graph, new_nodes: int, m: int) -> nx.Graph:
    """
    Estende il grafo con nuovi nodi utilizzando l'attaccamento preferenziale.

    Args:
        undirected_graph (nx.Graph): Grafo non orientato da espandere.
        new_nodes (int): Numero di nuovi nodi da aggiungere.
        m (int): Numero di connessioni per ogni nuovo nodo.

    Returns:
        nx.Graph: Il grafo non orientato esteso.
    """
    G = deepcopy(undirected_graph)

    for _ in range(new_nodes):
        new_node = len(G.nodes)
        target_nodes = []

        while len(target_nodes) < m:
            degrees = dict(G.degree)
            total_degree = sum(degrees.values())
            probabilities = {node: degree / total_degree for node, degree in degrees.items()}

            target_node = random.choices(list(probabilities.keys()), weights = probabilities.values(), k = 1)[0]

            if target_node not in target_nodes:
                target_nodes.append(target_node)

        G.add_node(new_node)
        for target in target_nodes:
            G.add_edge(new_node, target)

    return G

new_nodes = 350
edges_per_node = 3

undirected_graph = graph_expand_preferential_attachment(directed_graph.to_undirected(), new_nodes, edges_per_node)

## Per ciascuno dei due grafi

* Produrre una visualizzazione statica del grafo:
  * Utilizzare il Layout di Fruchterman-Reingold.

In [ ]:
def visualize_static_graph(graph: nx.Graph | nx.DiGraph, title: str = "Dependency Graph") -> None:
    """
    Produce una visualizzazione statica del grafo.

    Args:
        graph (nx.Graph | nx.DiGraph): Grafo orientato o non orientato da visualizzare.
        title (str): Titolo della visualizzazione.

    Returns:
        None: La funzione produce un grafo con matplotlib.pyplot.
    """
    pos = nx.spring_layout(graph)
    plt.figure(figsize = (12, 12))
    nx.draw(graph, pos, with_labels = True, node_size = 800, font_size = 12)
    plt.title(title)
    plt.show()

def visualize_FR_graph(graph: nx.Graph, title = "Fruchterman-Reingold Graph") -> None:
    """
    Produce una visualizzazione statica del grafo con il layout di Fruchterman-Reingold.

    Args:
        graph (nx.Graph): Grafo non orientato da visualizzare.
        title (str, optional): Titolo della visualizzazione.

    Returns:
        None: La funzione produce un grafo con matplotlib.pyplot.
    """
    pos = nx.fruchterman_reingold_layout(graph, )
    plt.figure(figsize = (12, 12))
    nx.draw(graph, pos, with_labels = True, node_size = 800, font_size = 12)
    plt.title(title)
    plt.show()

In [ ]:
# Per il grafo indiretto utilizziamo lo spring layout,
# in quanto quello di Fruchterman-Reingold produce
# un grafo diretto

visualize_FR_graph(directed_graph)
visualize_static_graph(undirected_graph)

Output hidden; open in https://colab.research.google.com to view.

## Utilizzando PyVis, produrre una visualizzazione interattiva per ciascuno dei due grafi

In [ ]:
def visualize_with_pyvis(graph: nx.Graph | nx.DiGraph, output_filename: str = "html/graph.html") -> None:
    """
    Produce una visualizzazione interattiva del grafo usando PyVis, in cui:

    - La dimensione di ciascun nodo dipende dal numero di archi in ingresso al nodo
    - Il colore di ciascun nodo viene determinato secondo i criteri:
        - Grigio per i nodi appartenenti al primo quartile della distribuzione dei gradi
        - Blu per i nodi nel secondo quartile
        - Viola per i nodi nel terzo quartile
        - Giallo per i nodi nel quarto quartile

    Args:
        graph (nx.Graph | nx.DiGraph): il grafo da visualizzare.
        output_filename (str): Il nome del file HTML di output.

    Returns:
        None: La funzione scrive il file HTML.
    """
    net = Network(height = "800px", width = "100%", directed = graph.is_directed())

    # Calcolo dei gradi dei nodi (in-degree per i grafi diretti)
    if graph.is_directed():
        in_degrees = dict(graph.in_degree())
    else:
        in_degrees = dict(graph.degree())

    # Calcola Quartili
    degrees_list = list(in_degrees.values())
    q1 = np.percentile(degrees_list, 25)
    q2 = np.percentile(degrees_list, 50)
    q3 = np.percentile(degrees_list, 75)

    def get_color(degree) -> str:
        """
        Mappa i colori in base al quartile di appartenenza.

        Args:
            degree (dict): Dizionario contenente i nodi e i relativi gradi.

        Returns:
            str: stringa che rappresenta il colore del nodo.
        """
        if degree <= q1:
            return "gray"
        elif degree <= q2:
            return "blue"
        elif degree <= q3:
            return "purple"
        else:
            return "#E6B000FF"

    # Aggiunta dei nodi con dimensioni e colori
    for node, degree in in_degrees.items():
        net.add_node(
            node,
            labe = node,
            size = 15 + degree * 3.5,
            color = get_color(degree)
        )

    for edge in graph.edges:
        net.add_edge(edge[0], edge[1], color = "#D3D3D3")

    net.toggle_physics(False)
    net.set_options(
        """
            const options = {
                "physics": {
                    "enabled": true,
                    "barnesHut": {
                        "theta": 0.6,
                        "gravitationalConstant": -35772,
                        "centralGravity": 0,
                        "springLength": 310,
                        "springConstant": 0.03
                    },
                    "minVelocity": 0.75
                }
            }
        """
    )

    try:
      net.write_html(output_filename)
    except FileNotFoundError:
      print("File non trovato")

In [ ]:
# Grafo diretto
visualize_with_pyvis(directed_graph, "html/directed_graph.html")

# Grafo indiretto
visualize_with_pyvis(undirected_graph, "html/undirected_graph.html")

File non trovato
File non trovato


## Serializzare opportunamente i due grafi creati

In [ ]:
directed_graph_serialized = nx.node_link_data(directed_graph, edges = "edges")
undirected_graph_serialized = nx.node_link_data(undirected_graph, edges = "edges")

save_in_file(f"graphs/{directed_graph_serialized}.json", directed_graph_serialized)
save_in_file(f"graphs/{undirected_graph_serialized}.json", undirected_graph_serialized)

File non salvato
File non salvato


## Analisi dei dati

Una volta prodotti i due grafi e le loro visualizzazioni secondo le istruzioni presentate precedentemente, si dovrà svolgere un'attività di analisi dei dati secondo le istruzioni
seguenti.

1. Calcolare le seguenti misure per ciascuno dei due grafi:
  * Centro
  * Raggio
  * Distanza media
  * Distanza massima
  * Coefficiente di clustering medio
  * Transitività

2. Calcolare le seguenti misure di centralità per i nodi dei due grafi:
  * Betweenness centrality,
  * Closeness centrality,
  * Degree centrality,
  * In-degree centrality,
  * Out-degree centrality,
  * Page rank.

3. Calcolare i seguenti coefficienti per stimare la "small-worldness" dei grafi:
  * Coefficiente omega,
  * Coefficiente sigma.

4. Serializzare correttamente le misure nel formato descritto nel seguente JSON.

In [ ]:
def metrics_serializer(graph: nx.Graph | nx.DiGraph) -> dict:
    """
    Produce una serializzazione del grafo, contenente le metriche globali:

    - Centro
    - Raggio
    - Distanza media
    - Distanza massima
    - Coefficiente di clustering
    - Transitivita'

    E le metriche riguardo la centralita' per ogni nodo del grafo:

    - Betweenness centrality
    - Closeness centrality
    - Degreee centrality
    - PageRank

    NOTE: Non e' possibile calcolare le prime 4 metriche globali per i grafi diretti.

    Args:
        graph (nx.Graph | nx.DiGraph): Grafo rappresentante le dipendenze del pacchetto

    Returns:
        dict: Dizionario contenente le metriche globali e quelle per ogni nodo.
    """
    if graph.is_directed():
        directed = True
    else:
        directed = False

    nodes_centrality_measures = {}

    betweenness_centrality = nx.betweenness_centrality(graph)
    closeness_centrality = nx.closeness_centrality(graph)
    degree_centrality = nx.degree_centrality(graph) if not directed else None
    in_degree_centrality = nx.in_degree_centrality(graph) if directed else None
    out_degree_centrality = nx.out_degree_centrality(graph) if directed else None
    pagerank = nx.pagerank(graph, max_iter = 100)

    for node_name in list(graph.nodes):
        nodes_centrality_measures[node_name] = {
                "betweenness_centrality": betweenness_centrality[node_name],
                "closeness_centrality": closeness_centrality[node_name],
                "degree_centrality": degree_centrality[node_name] if not directed else None,
                "in_degree_centrality": in_degree_centrality[node_name] if directed else None,
                "out_degree_centrality": out_degree_centrality[node_name] if directed else None,
                "pagerank": pagerank[node_name]
            }

    seed = 123
    measures = {
        "global_matrics" : {
            "graph_center": nx.center(graph) if not directed else None,
            "radius": nx.radius(graph) if not directed else None,
            "average_distance": nx.diameter(graph) / nx.number_of_nodes(graph) if not directed else None,
            "max_distance": nx.diameter(graph) if not directed else None,
            "clustering_coefficient": nx.average_clustering(graph),
            "transitivity": nx.transitivity(graph)
        },
        "centrality_measures": nodes_centrality_measures,
        "small-worldness": {
            "omega": nx.omega(graph, 1, 1, seed) if not directed else None,
            "sigma": nx.sigma(graph, 1, 1, seed) if not directed else None
        }
    }

    return measures

Per ridurre il tempo di esecuzione, impostiamo il numero di iterazioni (niter) e di grafi generati randomicamente (nrand), delle funzioni omega e sigma, a 1. Questo operazione, però, riduce la precisione dei calcoli.

In [ ]:
save_in_file("data/undirected_graph_metrics.json", metrics_serializer(undirected_graph))
save_in_file("data/directed_graph_metrics.json", metrics_serializer(directed_graph))